# MCT4052 Workshop 6f: Repeated K-Fold Cross Validation

*Author: Stefano Fasciani, stefano.fasciani@imv.uio.no, Department of Musicology, University of Oslo.*

In notebook we use the [RepeatedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedKFold.html) and [RepeatedStratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html) to further improve the estimation of the model performances via cross validation. In particular Repeated K-Fold is likely to reduce the bias in the model’s estimated performance (although it may increase the variance) because the split in multiple time is performed randomly multiple time (and at each time a cross validation is performed). This reduced the likelyhood to have a particularly favorable split in our cross validation.

Stratification is the process of rearranging the data as to ensure each fold is a good representation of the whole set. For example in a binary classification problem where each class comprises 50% of the data, it is best to arrange the data such that in every fold, each class comprises around half the instances. The repeated stratified k-fold take care of this aspect. Mind that the stratification process make sense and it can be applied only to classification problems, while the repeated k-fold object works with both classification and regression problems. 

In this notebook we demonstrate how to apply repeated k-fold cross validation to a regression task, and how to apply stratified repeated k-fold to a classification task.


In [1]:
import numpy as np
import pandas as pd
import librosa
import sklearn
import os

### 1. Regression task with repeated k-fold cross validation

In [2]:
#loading files and computing features
sr = 22050

def extract_features_target(filename, sr):
    
    signal, dummy = librosa.load(filename, sr=sr, mono=True)
    output = librosa.feature.melspectrogram(y=signal, n_mels=25)
    output = output.flatten()
    
    #preparing the output array
    target = np.zeros((1,2))
    target[0,0] = np.mean(librosa.feature.rms(y=signal))
    target[0,1] = np.mean(librosa.feature.spectral_flatness(y=signal))
    
    return output, target

filenames = os.listdir('./data/examples3')
features = np.zeros((len(filenames),4325))
target = np.zeros((len(filenames),2))

for i in range(len(filenames)):
    features[i,:], target[i,:] = extract_features_target('./data/examples3/'+filenames[i], sr=sr)

print('Done!')

Done!


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold


#creating pipeline
pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('dim_red', PCA(n_components = 5)),
        ('classifier', MLPRegressor(hidden_layer_sizes=(12,8,4), max_iter=2000, activation='tanh'))
        ])

#creating the repeated k-fold, use random_state to get repeatable results
#with n_splits=5 we partition the data into 5 splits of 20% and use 4 for trainign and 1 for testing.
#the n_repeats indicates how many times the k-fold has to be repeated
rkf = RepeatedKFold(n_splits=5, n_repeats=10)

#initializing the cross validator with pipe, features, target, scores, and kfold object
scores = sklearn.model_selection.cross_validate(pipe, features, target, cv=rkf, scoring=('r2', 'neg_mean_squared_error'),return_train_score=True)

print(scores,'\n')

print('MSE mean and variance', np.mean(scores['test_neg_mean_squared_error']),np.var(scores['test_neg_mean_squared_error']),'\n')
print('R2 mean and variance', np.mean(scores['test_r2']),np.var(scores['test_r2']),'\n')


{'fit_time': array([0.09958816, 0.11857796, 0.06680107, 0.08196497, 0.0980742 ,
       0.07242894, 0.06333113, 0.09084606, 0.06792498, 0.1081841 ,
       0.10718489, 0.08938599, 0.11609221, 0.08312607, 0.06014395,
       0.10213804, 0.07937002, 0.06887603, 0.0767529 , 0.08306408,
       0.08760786, 0.09593391, 0.09061193, 0.10667205, 0.09140992,
       0.0826292 , 0.0798018 , 0.09002018, 0.12793803, 0.1116991 ,
       0.07158327, 0.09427714, 0.0783298 , 0.09697104, 0.07796669,
       0.11867189, 0.07640576, 0.10308766, 0.07693791, 0.09120703,
       0.11785007, 0.07044935, 0.10144496, 0.09703183, 0.09077883,
       0.07322598, 0.12408805, 0.08609271, 0.08992219, 0.08427501]), 'score_time': array([0.00256801, 0.00114608, 0.00145912, 0.00135779, 0.001127  ,
       0.00135684, 0.00134516, 0.00105596, 0.00114608, 0.00103998,
       0.00150919, 0.00216222, 0.00144792, 0.00262475, 0.00130987,
       0.00115108, 0.00246811, 0.00136495, 0.00120187, 0.00153708,
       0.00175905, 0.00629878, 0.

### 3. Classification task with stratified repeated k-fold cross validation

In [5]:
#loading files and extracting features
metadata = pd.read_csv('./data/examples4/meta.csv')
classes = list(metadata.label.unique())
print('There are',len(classes),'different classes:',classes)

sr = 22050

def extract_features(filename, sr):
    signal, dummy = librosa.load(filename, sr=sr, mono=True)
    output = np.mean(librosa.feature.mfcc(y=signal, n_mfcc=20), axis=1)
    return output

print('number of files in database',len(metadata.index))
features = np.zeros((len(metadata.index),20))
labels = np.zeros((len(metadata.index)))

for i, row in metadata.iterrows():
    features[i,:] = extract_features('./data/examples4/'+row['filename'], sr=sr)
    labels[i] = (classes.index(row['label']))

print('Done!')

There are 5 different classes: ['cello', 'guitar', 'clarinet', 'flute', 'harmonica']
number of files in database 60
Done!


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold

#creating pipeline
pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('dim_red', PCA(n_components = 10)),
        ('classifier', MLPClassifier(hidden_layer_sizes=(20,5), max_iter=10000, activation='relu'))
        ])

#creating the repeated stratified k-fold, use random_state to get repeatable results
#with n_splits=5 we partition the data into 5 splits of 20% and use 4 for trainign and 1 for testing.
#the n_repeats indicates how many times the k-fold has to be repeated
rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

#initializing and running the cross validator with pipe, features, labels, scores, and kfold object
scores = sklearn.model_selection.cross_validate(pipe, features, labels, cv=rkf, scoring=('f1_macro', 'accuracy'),return_train_score=True)

print(scores,'\n')
print('Accuracy mean and variance', np.mean(scores['test_accuracy']),np.var(scores['test_accuracy']),'\n')
print('F1 macro mean and variance', np.mean(scores['test_f1_macro']),np.var(scores['test_f1_macro']),'\n')


{'fit_time': array([0.10126209, 0.12444115, 0.09971666, 0.12416816, 0.10993004,
       0.10632396, 0.20066309, 0.14769721, 0.09562993, 0.09040999,
       0.16533303, 0.14000702, 0.090801  , 0.09310818, 0.095433  ,
       0.12820911, 0.14157891, 0.09063315, 0.16104221, 0.09823465,
       0.09812093, 0.10025096, 0.1282618 , 0.085078  , 0.0940249 ,
       0.11311579, 0.11833191, 0.10253   , 0.09027982, 0.21474314,
       0.11411381, 0.09425783, 0.11598301, 0.10563993, 0.08994269,
       0.16752005, 0.08140182, 0.12022901, 0.19190788, 0.12961197,
       0.09372711, 0.09198117, 0.08530688, 0.14935994, 0.10778403,
       0.11158705, 0.11296797, 0.13934612, 0.16433215, 0.14914298]), 'score_time': array([0.00067091, 0.00056911, 0.00059819, 0.00061798, 0.00061607,
       0.0005641 , 0.00059271, 0.0005939 , 0.00058508, 0.00059891,
       0.00061488, 0.00061512, 0.00057292, 0.00060081, 0.00060201,
       0.00057197, 0.00060701, 0.00058103, 0.00058103, 0.00054932,
       0.00057101, 0.00058103, 0.

### 4. Follow up activity

Apply the repeated k fold and stretified repeated k fold to classification and regression on ML applications you previously developed using both your databases. It is recommended to use pipelines.